<a href="https://colab.research.google.com/github/KIMNAMHYEON-Kpass/love-letter-lab/blob/main/notebooks/01_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 연애편지 생성을 위한 LLM 프롬프트 엔지니어링

이 노트북은 Hugging Face의 언어 모델을 사용하여 주어진 조건에 맞는 연애편지를 생성하는 과정을 담고 있습니다. 
실행 순서는 다음과 같습니다.
1. **설치**: 필요한 라이브러리를 설치합니다.
2. **모델 로드**: SKT-AI의 KoGPT2 모델을 불러옵니다.
3. **프롬프트 정의**: 편지 내용의 가이드라인이 될 프롬프트를 설정합니다.
4. **생성**: 정의된 프롬프트를 기반으로 모델이 편지를 생성합니다.
5. **검증**: 생성된 편지가 요구사항을 충족하는지 확인합니다.

### 1. 설치 (Installation)

In [ ]:
!pip install transformers accelerate sentencepiece

### 2. 모델 로드 (Model Loading)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

### 3. 프롬프트 정의 (Prompt Definition)

In [ ]:
# 편지 생성 프롬프트 변수 정의
name = "민수"
relation = "동기"
situation = "발표를 앞두고 긴장한 상태"
tone = "달달"
length = 3  # 생성할 문장 수

prompt = f"""
당신은 연애 편지 작가입니다.
[제약 조건]
- 반드시 존댓말로 작성합니다.
- 과장된 비유나 지나친 감정 표현을 피합니다.
- 톤은 {tone}로 유지합니다.
- 총 {length} 문장으로 작성합니다.
- 느낌표는 최대 1번만 사용합니다.
- 이모지는 사용하지 않습니다.
[상황 정보]
- 받는 사람 이름: {name}
- 관계: {relation}
- 상황: {situation}
위 정보를 모두 반영하여 자연스럽고 따뜻한 연애 편지를 작성하세요.
편지는 자연스러운 문단으로 출력하고, 인사말이나 서명은 넣지 마세요.
"""
print("--- 프롬프트 미리보기 ---")
print(prompt)

### 4. 생성 (Generation)

In [ ]:
# 하이퍼파라미터를 조절해 생성
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

# 문장당 30~50 토큰으로 가정하여 최대 토큰 수 설정
max_tokens = length * 50

output_sequences = model.generate(
    input_ids=input_ids,
    max_new_tokens=max_tokens,
    temperature=0.8,
    top_p=0.9,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

# 생성된 텍스트에서 프롬프트 부분 제거
full_generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
generated_letter = full_generated_text.replace(prompt, "").strip()

print("--- 생성된 편지 ---")
print(generated_letter)

### 5. 검증 (Verification)

In [ ]:
# 생성 결과 검증
print("--- 생성 결과 검증 ---")

# 검증 항목 1: 문장 수
# 간단하게 마침표(.) 개수로 문장 수를 근사합니다.
num_sentences = generated_letter.count('.') + generated_letter.count('!') + generated_letter.count('?')
print(f"요구 문장 수: {length}")
print(f"생성 문장 수: {num_sentences}")
if num_sentences >= length:
    print("-> 문장 수 검증: [PASS]")
else:
    print("-> 문장 수 검증: [FAIL]")

# 검증 항목 2: 금칙어 포함 여부
forbidden_words = ["이모지", "스맥다운", "게임"] # 프롬프트에 금지된 단어 및 생성 시 나타난 부적절한 단어
found_forbidden = False
for word in forbidden_words:
    if word in generated_letter:
        print(f"-> 금칙어 '{word}' 발견: [FAIL]")
        found_forbidden = True
        break
if not found_forbidden:
    print("-> 금칙어 검증: [PASS]")

# 검증 항목 3: 이름 포함 여부
if name in generated_letter:
    print(f"-> 이름 '{name}' 포함 검증: [PASS]")
else:
    print(f"-> 이름 '{name}' 포함 검증: [FAIL]")